# Notebook for replicating the analyses in Tennessee Business Tax Reform 2024
### Richard W. Evans, July 2024
This notebook replicates the analyses in the article by [Richard W. Evans](https://sites.google.com/site/rickecon).

## 1. Introduction
Before opening and running this notebook, make sure that you have downloaded or cloned the [`TN-BusTax2024`](https://github.com/OpenSourceEcon/TN-BusTax2024) repository (https://github.com/OpenSourceEcon/TN-BusTax2024) and created and activated the associated conda environment `tn-bustax2024-dev` in the `environment.yml` file. You can also run this notebook easily in the cloud by going to this [Google Colab version](https://colab.research.google.com/drive/1dDKVTEAm4drvNXgHpm617j-82X4xzxKh?usp=sharing) of this notebook.

In [2]:
# import packages
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import (
    ColumnDataSource, Title, Label, LabelSet, Legend, LegendItem,
    CategoricalColorMapper, ColorBar, HoverTool, NumeralTickFormatter,
    GeoJSONDataSource, FactorRange
)
from bokeh.models.tickers import SingleIntervalTicker
from bokeh.transform import factor_cmap
from bokeh.sampledata.us_states import data as states
from bokeh.transform import cumsum
from bokeh.palettes import Category20c

import random
import json

### 1.1. Figure 1. Rainy Day Fund and Total Reserves as Percentages of General-Fund Expenditures, Tennessee and 50-State Median: 2000-2023

Get the Pew state rainy day fund data. This does not include District of Columbia. The 2022 data is now final. The 2023 data is labeled as "Enacted" which means projected based on enacted legislation.

In [ ]:
# Read in the Pew data on rainy day funds by year and state from worksheet
rain_totbal_path = (
    "https://github.com/OpenSourceEcon/TN-BusTax2024/blob/main/data/" +
    "ReservesBalancesData.xlsx?raw=true"
)
rain_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Rainy Day Fund Data",
    header=4,
    index_col=0,
    skipfooter=2
)

# Create a DataFrame of just the rainy day funds in $millions
# by state and by year
rain_dol_df = rain_df.loc[:'Wyoming', 'FY 2000.1':'FY 2023 Estimated']
rain_dol_df = rain_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016': '2016',
    ' FY 2017': '2017',
    ' FY 2018': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY2022': '2022',
    'FY 2023 Estimated': '2023'
})
rain_dol_df

# Create a DataFrame of just the rainy day funds as a percent of
# general fund expenditures by state and by year
rain_pct_df = rain_df.loc[:, 'FY 2000.2':'FY 2023 Estimated.1']
rain_pct_df = rain_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY2022.1': '2022',
    'FY 2023 Estimated.1': '2023'
})

In [ ]:
# Read in the PEW data on total reserves and balances by year and state from
# worksheet
totbal_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Total Balances Data",
    header=5,
    index_col=0,
    skipfooter=2
)

# Create a DataFrame of just the total reserves and balances funds in $millions
# by state and by year
totbal_dol_df = totbal_df.loc[:'Wyoming', 'FY 2000.1':'FY 2023 Estimated.1']
totbal_dol_df = totbal_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY2022.1': '2022',
    'FY 2023 Estimated.1': '2023'
})

# Create a DataFrame of just the total reserves and balances funds as percent
# of general fund expenditures by state and by year
totbal_pct_df = totbal_df.loc[:, 'FY 2000.2':'FY 2023 Estimated.2']
totbal_pct_df = totbal_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY2022.2': '2022',
    'FY 2023 Estimated.2': '2023'
})

In [ ]:
rain_pct_50_df = rain_pct_df.loc["50-state median", :].to_frame().reset_index()
rain_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}, inplace = True
)
rain_pct_50_df["percent"] = 100 * rain_pct_50_df["fraction"]
rain_pct_50_df["series"] = "50-state median Rainy Day Fund"

rain_pct_tn_df = rain_pct_df.loc["Tennessee", :].to_frame().reset_index()
rain_pct_tn_df = rain_pct_tn_df.rename(
    columns = {"index":"year", "Tennessee":"fraction"}
)
rain_pct_tn_df["percent"] = 100 * rain_pct_tn_df["fraction"]
rain_pct_tn_df["series"] = "Tennessee Rainy Day Fund"

totbal_pct_50_df = \
    totbal_pct_df.loc["50-state median", :].to_frame().reset_index()
totbal_pct_50_df = totbal_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}
)
totbal_pct_50_df["percent"] = 100 * totbal_pct_50_df["fraction"]
totbal_pct_50_df["series"] = "50-state median total reserve balances"

totbal_pct_tn_df = totbal_pct_df.loc["Tennessee", :].to_frame().reset_index()
totbal_pct_tn_df = totbal_pct_tn_df.rename(
    columns = {'index':'year', "Tennessee":"fraction"}
)
totbal_pct_tn_df["percent"] = 100 * totbal_pct_tn_df["fraction"]
totbal_pct_tn_df["series"] = "Tennessee total reserve balances"

# Merge the four DataFrames to save the joint dataframe as a .csv
fig1_source_df = rain_pct_50_df[['year', 'percent']].rename(
    columns={'percent':'rain_50m_pct'}
)
fig1_source_df = pd.merge(
    fig1_source_df,
    rain_pct_tn_df[['year', 'percent']].rename(
        columns={'percent':'rain_tn_pct'}
    ),
    on='year', how='inner'
)
fig1_source_df = pd.merge(
    fig1_source_df,
    totbal_pct_50_df[['year', 'percent']].rename(
        columns={'percent':'totbal_50m_pct'}
    ),
    on='year', how='inner'
)
fig1_source_df = pd.merge(
    fig1_source_df,
    totbal_pct_tn_df[['year', 'percent']].rename(
        columns={'percent':'totbal_tn_pct'}
    ),
    on='year', how='inner'
)
fig1_source_df.to_csv('./data/fig1_source.csv', index=False)

fig1_source_df

### PEW state balances data: Rainy day fund balances

In [2]:
# Read in the PEW data on rainy day funds by year and
# state from worksheet
rain_totbal_path = (
    "https://github.com/TheCGO/TN-BusinessTax/blob/main/data/" +
    "ReservesBalancesData.xlsx?raw=true"
)
rain_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Rainy Day Fund Data",
    header=5,
    index_col=0,
    skipfooter=17
)
rain_df.replace(0, np.nan, inplace=True)

# Create a DataFrame of just the rainy day funds in $millions
# by state and by year
rain_dol_df = rain_df.loc[:'Wyoming', 'FY 2000.1':'FY 2022 (estimated).1']
rain_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY 2022 (estimated).1': '2022',
}, inplace = True)

# Create a DataFrame of just the rainy day funds as a percent of
# general fund expenditures by state and by year
rain_pct_df = rain_df.loc[:, 'FY 2000.2':'FY 2022 (estimated).2']
rain_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY 2022 (estimated).2': '2022',
}, inplace = True)

/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_56874/3442603919.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_pct_df.rename(columns = {


### PEW state balances data: Total balances

In [3]:
# Read in the PEW data on total reserves and balances
# by year and state from worksheet
totbal_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Total Balances Data",
    header=5,
    index_col=0,
    skipfooter=17
)
totbal_df.replace(0, np.nan, inplace=True)

# Create a DataFrame of just the total reserves and balances
# funds in $millions by state and by year
totbal_dol_df = totbal_df.loc[:'Wyoming', 'FY 2000.1':'FY 2022 (estimated).1']
totbal_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY 2022 (estimated).1': '2022',
}, inplace = True)

# Create a DataFrame of just the total reserves and balances funds
# as a percent of general fund expenditures by state and by year
totbal_pct_df = totbal_df.loc[:, 'FY 2000.2':'FY 2022 (estimated).2']
totbal_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY 2022 (estimated).2': '2022',
}, inplace = True)

/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_56874/3735137075.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totbal_pct_df.rename(columns = {


### Figure 1. Plot time series of rainy day funds and total balances as percent of general fund expenditures for both the 50-state median and for Tennessee
Executing the cell below and the following cell will create the `rain_totbal_50_tn_timeseries.html` file in this notebook and in the `/images/` folder of this repository. The figure should also open in a browser window in your browser. I created the `.png` version of the file by just screenshotting the `.html` image.

In [13]:
# Create source DataFrame for Figure 1 and save it as .csv
rain_pct_50_df = rain_pct_df.loc["50-state median", :].to_frame().reset_index()
rain_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}, inplace = True
)
rain_pct_50_df["percent"] = 100 * rain_pct_50_df["fraction"]

rain_pct_tn_df = rain_pct_df.loc["Tennessee", :].to_frame().reset_index()
rain_pct_tn_df.rename(
    columns = {"index":"year", "Tennessee":"fraction"}, inplace = True
)
rain_pct_tn_df["percent"] = 100 * rain_pct_tn_df["fraction"]

totbal_pct_50_df = \
    totbal_pct_df.loc["50-state median", :].to_frame().reset_index()
totbal_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}, inplace = True
)
totbal_pct_50_df["percent"] = 100 * totbal_pct_50_df["fraction"]

totbal_pct_tn_df = totbal_pct_df.loc["Tennessee", :].to_frame().reset_index()
totbal_pct_tn_df.rename(
    columns = {'index':'year', "Tennessee":"fraction"}, inplace = True
)
totbal_pct_tn_df["percent"] = 100 * totbal_pct_tn_df["fraction"]

# Merge the four DataFrames to save the joint dataframe as a .csv
fig1_source_df = rain_pct_50_df[['year', 'percent']].rename(
    columns={'percent':'rain_50m_pct'}
)
fig1_source_df = pd.merge(
    fig1_source_df,
    rain_pct_tn_df[['year', 'percent']].rename(
        columns={'percent':'rain_tn_pct'}
    ),
    on='year', how='inner'
)
fig1_source_df = pd.merge(
    fig1_source_df,
    totbal_pct_50_df[['year', 'percent']].rename(
        columns={'percent':'totbal_50m_pct'}
    ),
    on='year', how='inner'
)
fig1_source_df = pd.merge(
    fig1_source_df,
    totbal_pct_tn_df[['year', 'percent']].rename(
        columns={'percent':'totbal_tn_pct'}
    ),
    on='year', how='inner'
)
fig1_source_df.to_csv('data/fig1_source.csv', index=False)
fig1_source_df

,year,rain_50m_pct,rain_tn_pct,totbal_50m_pct,totbal_tn_pct
0,2000,4.127259,2.504361,8.766520,3.285552
1,2001,4.614270,2.537384,7.148315,2.983564
2,2002,1.655668,2.368375,2.887308,2.521389
3,2003,0.722015,2.249264,2.656212,3.059252
4,2004,1.844196,2.654402,5.246650,9.323433
5,2005,2.459819,3.022189,9.016673,8.094287
6,2006,4.553273,3.585613,11.801597,11.793910
7,2007,4.738918,5.553680,11.327766,15.849829
8,2008,4.834502,6.835208,8.292193,10.004921
9,2009,2.728263,5.212919,4.582794,5.936077


In [14]:
# fig1_title = ("Rainy Day fund and total reserves as a percentage of " +
#               "general fund expenditures: 2000-2022")
fig1_title = ""
output_file("images/rain_totbal_50_tn_timeseries.html", title=fig1_title,
            mode='inline')
output_notebook()
min_year = 2000
max_year = 2022
min_pct = 2.0
max_pct = 0.0
pct_buffer = 0.05

var_list = [rain_pct_50_df, rain_pct_tn_df, totbal_pct_50_df, totbal_pct_tn_df]
color_list = ["#3477A5", "#D5AB53", "#3477A5", "#D5AB53"]
marker_list = ["circle", "circle", "square", "square"]
legend_label_list = [
    "Rainy Day Fund, 50-state median", "Rainy Day Fund, Tennessee",
    "Total balances, 50-state median", "Total balances, Tennessee"
]
cds_list = []

for k, df in enumerate(var_list):
    min_pct = np.minimum(min_pct, df["percent"].min())
    max_pct = np.maximum(max_pct, df["percent"].max())
    cds_list.append(ColumnDataSource(df))

fig1 = figure(title=fig1_title,
              height=420,
              width=700,
              x_axis_label='Year',
              x_range=(min_year - 1, max_year + 1.5),
              y_axis_label='Percent of general fund expenditures',
              y_range=(min_pct - pct_buffer * (max_pct - min_pct),
                       max_pct + pct_buffer * (max_pct - min_pct)),
              toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig1.toolbar.active_drag = None
fig1.toolbar.active_scroll = None
fig1.toolbar.active_tap = None

# Set title font size and axes font sizes
fig1.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig1.title.text_color = '#434244'
fig1.xaxis.axis_label_text_font_size = '12pt'
fig1.xaxis.major_label_text_font_size = '12pt'
fig1.xaxis.major_label_text_color = '#434244'
fig1.yaxis.axis_label_text_font_size = '12pt'
fig1.yaxis.major_label_text_font_size = '12pt'
fig1.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig1.xaxis.ticker = SingleIntervalTicker(interval=2, num_minor_ticks=2)
fig1.xaxis.axis_line_color = '#434244'
fig1.xaxis.major_tick_line_color = '#434244'
fig1.xaxis.minor_tick_line_color = '#434244'
fig1.xgrid.ticker = SingleIntervalTicker(interval=2)
fig1.yaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig1.yaxis.axis_line_color = '#434244'
fig1.yaxis.major_tick_line_color = '#434244'
fig1.yaxis.minor_tick_line_color = '#434244'
fig1.ygrid.ticker = SingleIntervalTicker(interval=10)

# Create lines and markers for time series
for k, yvar in enumerate(var_list):
    fig1.line(x='year', y='percent', source=cds_list[k], color=color_list[k],
              line_width=3, alpha=0.7)
    fig1.scatter(x='year', y='percent', source=cds_list[k], size=8,
                 line_width=1, line_color='black', fill_color=color_list[k],
                 marker=marker_list[k], line_alpha=0.7, fill_alpha=0.7,
                 legend_label=legend_label_list[k])

fig1.segment(x0=2021.4, y0=min_pct - pct_buffer * (max_pct - min_pct),
             x1=2021.4, y1=max_pct + pct_buffer * (max_pct - min_pct),
             color='#434244', line_dash='6 2', line_width=2)

label_temp = Label(x=2021.5, y=28.0, x_units='data', y_units='data',
                   text='Projected', text_font_size='9pt',
                   text_color='#434244')
fig1.add_layout(label_temp)

# Add information on hover
tooltips = [('Year', '@year'),
            ('Pct of gen. fund exps.','@percent{0.0}' + '%')]
fig1.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig1.legend.location = 'top_center'
fig1.legend.border_line_width = 1
fig1.legend.border_line_color = '#434244'
fig1.legend.border_line_alpha = 1
fig1.legend.label_text_font_size = '12pt'
fig1.legend.label_text_color = '#434244'

# Add notes below image
note_text_list1 = [
    (
        'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and ' +
        'Analysis," December 16, 2022,'
    ),
    ('        accessed December 31, 2022.')
]
for note_text in note_text_list1:
    caption = Title(
        text=note_text, align='left', text_font_size='11pt',
        text_font_style='normal',
        text_color='#434244',
        # text_font='Open Sans'
    )
    fig1.add_layout(caption, 'below')

# # This export_png() function requires selenium package as well as firefox
# # and geckodriver packages
# # (see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/export.html)
# export_png(fig, filename="/images/rain_totbal_50_tn_timeseries.png")

Loading BokehJS ...

In [15]:
show(fig1)

### Create Figure 1 without the source text underneath (for web publication) 

In [16]:
output_file("images/rain_totbal_50_tn_timeseries_nosubtxt.html",
            title=fig1_title, mode='inline')
output_notebook()
min_year = 2000
max_year = 2022
min_pct = 2.0
max_pct = 0.0
pct_buffer = 0.05

var_list = [rain_pct_50_df, rain_pct_tn_df, totbal_pct_50_df, totbal_pct_tn_df]
color_list = ["#3477A5", "#D5AB53", "#3477A5", "#D5AB53"]
marker_list = ["circle", "circle", "square", "square"]
legend_label_list = [
    "Rainy Day Fund, 50-state median", "Rainy Day Fund, Tennessee",
    "Total balances, 50-state median", "Total balances, Tennessee"
]
cds_list = []

for k, df in enumerate(var_list):
    min_pct = np.minimum(min_pct, df["percent"].min())
    max_pct = np.maximum(max_pct, df["percent"].max())
    cds_list.append(ColumnDataSource(df))

fig1_nosubtxt = figure(title=fig1_title,
                       height=420,
                       width=700,
                       x_axis_label='Year',
                       x_range=(min_year - 1, max_year + 1.5),
                       y_axis_label='Percent of general fund expenditures',
                       y_range=(min_pct - pct_buffer * (max_pct - min_pct),
                                max_pct + pct_buffer * (max_pct - min_pct)),
                       toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig1_nosubtxt.toolbar.active_drag = None
fig1_nosubtxt.toolbar.active_scroll = None
fig1_nosubtxt.toolbar.active_tap = None

# Set title font size and axes font sizes
fig1_nosubtxt.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig1_nosubtxt.title.text_color = '#434244'
fig1_nosubtxt.xaxis.axis_label_text_font_size = '12pt'
fig1_nosubtxt.xaxis.major_label_text_font_size = '12pt'
fig1_nosubtxt.xaxis.major_label_text_color = '#434244'
fig1_nosubtxt.yaxis.axis_label_text_font_size = '12pt'
fig1_nosubtxt.yaxis.major_label_text_font_size = '12pt'
fig1_nosubtxt.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig1_nosubtxt.xaxis.ticker = SingleIntervalTicker(interval=2, num_minor_ticks=2)
fig1_nosubtxt.xaxis.axis_line_color = '#434244'
fig1_nosubtxt.xaxis.major_tick_line_color = '#434244'
fig1_nosubtxt.xaxis.minor_tick_line_color = '#434244'
fig1_nosubtxt.xgrid.ticker = SingleIntervalTicker(interval=2)
fig1_nosubtxt.yaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig1_nosubtxt.yaxis.axis_line_color = '#434244'
fig1_nosubtxt.yaxis.major_tick_line_color = '#434244'
fig1_nosubtxt.yaxis.minor_tick_line_color = '#434244'
fig1_nosubtxt.ygrid.ticker = SingleIntervalTicker(interval=10)

# Create lines and markers for time series
for k, yvar in enumerate(var_list):
    fig1_nosubtxt.line(x='year', y='percent', source=cds_list[k], color=color_list[k],
                       line_width=3, alpha=0.7)
    fig1_nosubtxt.scatter(x='year', y='percent', source=cds_list[k], size=8,
                          line_width=1, line_color='black', fill_color=color_list[k],
                          marker=marker_list[k], line_alpha=0.7, fill_alpha=0.7,
                          legend_label=legend_label_list[k])

fig1_nosubtxt.segment(x0=2021.4, y0=min_pct - pct_buffer * (max_pct - min_pct),
                      x1=2021.4, y1=max_pct + pct_buffer * (max_pct - min_pct),
                      color='#434244', line_dash='6 2', line_width=2)

label_temp = Label(x=2021.5, y=28.0, x_units='data', y_units='data',
                   text='Projected', text_font_size='9pt', text_color='#434244')
fig1_nosubtxt.add_layout(label_temp)

# Add information on hover
tooltips = [('Year', '@year'),
            ('Pct of gen. fund exps.','@percent{0.0}' + '%')]
fig1_nosubtxt.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig1_nosubtxt.legend.location = 'top_center'
fig1_nosubtxt.legend.border_line_width = 1
fig1_nosubtxt.legend.border_line_color = '#434244'
fig1_nosubtxt.legend.border_line_alpha = 1
fig1_nosubtxt.legend.label_text_font_size = '12pt'
fig1_nosubtxt.legend.label_text_color = '#434244'

# # Add notes below image
# for note_text in note_text_list1:
#     caption = Title(
#         text=note_text, align='left', text_font_size='11pt',
#         text_font_style='normal',
#         text_color='#434244',
#         # text_font='Open Sans'
#     )
#     fig1_nosubtxt.add_layout(caption, 'below')

# # This export_png() function requires selenium package as well as firefox
# # and geckodriver packages
# # (see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/export.html)
# export_png(fig, filename="/images/rain_totbal_50_tn_timeseries.png")

Loading BokehJS ...

In [17]:
show(fig1_nosubtxt)

### Figure 2. Estimated 2022 rainy day fund and total balances as percent of general fund expenditure by state in order of rainy day fund balances, highlighting Tennessee

In [18]:
rain_pct_2022_df = rain_pct_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
rain_pct_2022_df.rename(columns = {"index": "state", "2022": "rain_frac"}, inplace = True)
rain_pct_2022_df["rain_pct"] = 100 * rain_pct_2022_df["rain_frac"]
rain_pct_2022_df.drop('rain_frac', axis=1, inplace=True)

rain_dol_2022_df = rain_dol_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
rain_dol_2022_df.rename(columns = {"index": "state", "2022": "rain_dol"}, inplace = True)

totbal_pct_2022_df = totbal_pct_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
totbal_pct_2022_df.rename(columns = {"index": "state", "2022": "totbal_frac"}, inplace = True)
totbal_pct_2022_df["totbal_pct"] = 100 * totbal_pct_2022_df["totbal_frac"]
totbal_pct_2022_df.drop('totbal_frac', axis=1, inplace=True)

totbal_dol_2022_df = totbal_dol_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
totbal_dol_2022_df.rename(columns = {"index": "state", "2022": "totbal_dol"}, inplace = True)

# Merge the four DataFrames
rain_totbal_pct_dol_2022_df = rain_pct_2022_df.copy()
rain_totbal_pct_dol_2022_df = pd.merge(rain_totbal_pct_dol_2022_df, rain_dol_2022_df, on='state', how='inner')
rain_totbal_pct_dol_2022_df["rain_color"] = "#D5AB53"
rain_totbal_pct_dol_2022_df["rain_color"][rain_totbal_pct_dol_2022_df["state"]=="Tennessee"] = "#EED17E"
rain_totbal_pct_dol_2022_df = pd.merge(rain_totbal_pct_dol_2022_df, totbal_pct_2022_df, on='state', how='inner')
rain_totbal_pct_dol_2022_df = pd.merge(rain_totbal_pct_dol_2022_df, totbal_dol_2022_df, on='state', how='inner')
rain_totbal_pct_dol_2022_df["totbal_color"] = "#3477A5"
rain_totbal_pct_dol_2022_df["totbal_color"][rain_totbal_pct_dol_2022_df["state"]=="Tennessee"] = "#74AAC7"

# Create a new variable that is totbal_pct - rain_pct
rain_totbal_pct_dol_2022_df["totbal_rain_pct_dif"] = (
    rain_totbal_pct_dol_2022_df["totbal_pct"] - rain_totbal_pct_dol_2022_df["rain_pct"]
)

# Sort by rainy day fund percent
rain_totbal_pct_dol_2022_sorted_df = rain_totbal_pct_dol_2022_df.sort_values(
    by=['rain_pct', 'totbal_pct'], ascending=[True, True]
)

rain_totbal_pct_dol_2022_sorted_df.to_csv('data/fig2_source.csv', index=False)
rain_totbal_pct_dol_2022_sorted_df

/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_56874/2255531318.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_totbal_pct_dol_2022_df["rain_color"][rain_totbal_pct_dol_2022_df["state"]=="Tennessee"] = "#EED17E"
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_56874/2255531318.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_totbal_pct_dol_2022_df["totbal_color"][rain_totbal_pct_dol_2022_df["state"]=="Tennessee"] = "#74AAC7"


,state,rain_pct,rain_dol,rain_color,totbal_pct,totbal_dol,totbal_color,totbal_rain_pct_dif
46,Washington,1.139188,305.300000,#D5AB53,23.588609,6321.700000,#3477A5,22.449421
12,Illinois,1.348466,600.000000,#D5AB53,3.371165,1500.000000,#3477A5,2.022699
31,New York,3.887742,3351.000000,#D5AB53,35.400376,30513.000000,#3477A5,31.512634
25,Montana,4.395113,118.000000,#D5AB53,28.933476,776.805971,#3477A5,24.538363
19,Maryland,5.474238,1160.226358,#D5AB53,23.045000,4884.230000,#3477A5,17.570762
7,Delaware,5.504065,280.300000,#D5AB53,44.594117,2271.000000,#3477A5,39.090052
38,Rhode Island,5.565183,253.260983,#D5AB53,8.542891,388.770886,#3477A5,2.977708
8,Florida,6.344123,2729.800000,#D5AB53,32.600723,14027.700000,#3477A5,26.256600
15,Kansas,6.430386,600.000000,#D5AB53,17.553881,1637.900000,#3477A5,11.123496
17,Louisiana,7.160229,720.816069,#D5AB53,22.566079,2271.710000,#3477A5,15.405850


In [9]:
# fig2_title = ('Estimated 2022 Rainy day Fund Balances and Total Fund ' +
#               'Balances as Percent of General Fund Expenditures')
fig2_title = ""
output_file("images/rain_totbal_pct_2022.html", title=fig2_title, mode='inline')
output_notebook()

states = rain_totbal_pct_dol_2022_sorted_df['state'].tolist()
legend_label_list = ["Rainy day fund", "Total reserves and balances"]
rain_totbal_pct_dol_2022_cds = ColumnDataSource(rain_totbal_pct_dol_2022_sorted_df)

min_pct = rain_totbal_pct_dol_2022_sorted_df['rain_pct'].min()
max_pct = rain_totbal_pct_dol_2022_sorted_df['totbal_pct'].max()
pct_buffer = 0.05

fig2 = figure(title=fig2_title,
              height=900,
              width=550,
              y_range=states,
              x_axis_label='Percent of general fund expenditures',
              x_range=(min_pct - pct_buffer * (max_pct - min_pct),
                       max_pct + pct_buffer * (max_pct - min_pct)),
              toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig2.toolbar.active_drag = None
fig2.toolbar.active_scroll = None
fig2.toolbar.active_tap = None

# Set title font size and axes font sizes
fig2.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig2.title.text_color = '#434244'
fig2.xaxis.axis_label_text_font_size = '11pt'
fig2.xaxis.major_label_text_font_size = '11pt'
fig2.xaxis.major_label_text_color = '#434244'
fig2.yaxis.axis_label_text_font_size = '9pt'
fig2.yaxis.major_label_text_font_size = '9pt'
fig2.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig2.xaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig2.xaxis.axis_line_color = '#434244'
fig2.xaxis.major_tick_line_color = '#434244'
fig2.xaxis.minor_tick_line_color = '#434244'
fig2.xgrid.ticker = SingleIntervalTicker(interval=10)
fig2.yaxis.axis_line_color = '#434244'
fig2.yaxis.major_tick_line_color = '#434244'
fig2.yaxis.minor_tick_line_color = '#434244'

fig2.hbar_stack(
    ['rain_pct', 'totbal_rain_pct_dif'], y='state', height=0.9,
    color=['rain_color', 'totbal_color'],
    source=rain_totbal_pct_dol_2022_cds, legend_label=legend_label_list
)

# Add information on hover
tooltips = [('State', '@state'),
            ('Rainy day fund % exps.','@rain_pct{0.0}' + '%'),
            ('Total balances % exps.','@totbal_pct{0.0}' + '%'),
           ]
fig2.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig2.legend.location = 'center_right'
fig2.legend.border_line_width = 1
fig2.legend.border_line_color = '#434244'
fig2.legend.border_line_alpha = 1
fig2.legend.label_text_font_size = '12pt'
fig2.legend.label_text_color = '#434244'
fig2.y_range.range_padding = 0.02

# Add notes below image
note_text_list2 = [
    (
        'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and'
    ),
    (
        '        Analysis," December 16, 2022, accessed December 31, 2022.'
    ),
    (
        'Note: For states in which the blue bar is not visible for total ' +
        'balances'
    ),
    (
        '         and reserves, the value equals the rainy day fund balance'
    ),
    (
        '         percentage.'
    )
]
for note_text in note_text_list2:
    caption = Title(text=note_text, align='left', text_font_size='11pt',
                    text_font_style='normal', text_color='#434244')
    fig2.add_layout(caption, 'below')

Loading BokehJS ...

In [10]:
show(fig2)

### Create Figure 2 without the source text underneath (for web publication) 

In [11]:
output_file("images/rain_totbal_pct_2022_nosubtxt.html", title=fig2_title, mode='inline')
output_notebook()

states = rain_totbal_pct_dol_2022_sorted_df['state'].tolist()
legend_label_list = ["Rainy day fund", "Total reserves and balances"]
rain_totbal_pct_dol_2022_cds = ColumnDataSource(rain_totbal_pct_dol_2022_sorted_df)

min_pct = rain_totbal_pct_dol_2022_sorted_df['rain_pct'].min()
max_pct = rain_totbal_pct_dol_2022_sorted_df['totbal_pct'].max()
pct_buffer = 0.05

fig2_nosubtxt = figure(title=fig2_title,
                       height=900,
                       width=550,
                       y_range=states,
                       x_axis_label='Percent of general fund expenditures',
                       x_range=(min_pct - pct_buffer * (max_pct - min_pct),
                                max_pct + pct_buffer * (max_pct - min_pct)),
                       toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig2_nosubtxt.toolbar.active_drag = None
fig2_nosubtxt.toolbar.active_scroll = None
fig2_nosubtxt.toolbar.active_tap = None

fig2.toolbar.active_drag = None
fig2.toolbar.active_scroll = None
fig2.toolbar.active_tap = None

# Set title font size and axes font sizes
fig2_nosubtxt.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig2_nosubtxt.title.text_color = '#434244'
fig2_nosubtxt.xaxis.axis_label_text_font_size = '11pt'
fig2_nosubtxt.xaxis.major_label_text_font_size = '11pt'
fig2_nosubtxt.xaxis.major_label_text_color = '#434244'
fig2_nosubtxt.yaxis.axis_label_text_font_size = '9pt'
fig2_nosubtxt.yaxis.major_label_text_font_size = '9pt'
fig2_nosubtxt.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig2_nosubtxt.xaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig2_nosubtxt.xaxis.axis_line_color = '#434244'
fig2_nosubtxt.xaxis.major_tick_line_color = '#434244'
fig2_nosubtxt.xaxis.minor_tick_line_color = '#434244'
fig2_nosubtxt.xgrid.ticker = SingleIntervalTicker(interval=10)
fig2_nosubtxt.yaxis.axis_line_color = '#434244'
fig2_nosubtxt.yaxis.major_tick_line_color = '#434244'
fig2_nosubtxt.yaxis.minor_tick_line_color = '#434244'

fig2_nosubtxt.hbar_stack(
    ['rain_pct', 'totbal_rain_pct_dif'], y='state', height=0.9,
    color=['rain_color', 'totbal_color'],
    source=rain_totbal_pct_dol_2022_cds, legend_label=legend_label_list
)

# Add information on hover
tooltips = [('State', '@state'),
            ('Rainy day fund % exps.','@rain_pct{0.0}' + '%'),
            ('Total balances % exps.','@totbal_pct{0.0}' + '%'),
           ]
fig2_nosubtxt.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig2_nosubtxt.legend.location = 'center_right'
fig2_nosubtxt.legend.border_line_width = 1
fig2_nosubtxt.legend.border_line_color = '#434244'
fig2_nosubtxt.legend.border_line_alpha = 1
fig2_nosubtxt.legend.label_text_font_size = '12pt'
fig2_nosubtxt.legend.label_text_color = '#434244'
fig2_nosubtxt.y_range.range_padding = 0.02

# # Add notes below image
# for note_text in note_text_list2:
#     caption = Title(text=note_text, align='left', text_font_size='11pt',
#                     text_font_style='normal', text_color='#434244')
#     fig2_nosubtxt.add_layout(caption, 'below')

Loading BokehJS ...

In [12]:
show(fig2_nosubtxt)

### Table 1. Number of states for which estimated 2022 amounts represent 23-year high for select categories of rainy day fund and total balances and reserves statistics: 2000-2022

In [9]:
# Number of states for which estimated 2022 rainy day fund balances
# represent a 23-year high
rain_dol_df["max_2021"] = False
rain_dol_df["max_2022"] = False
rain_dol_df["max_2021"][
    rain_dol_df["2021"] > rain_dol_df.loc[:, "2000":"2020"].max(axis=1)
] = True
rain_dol_df["max_2022"][
    rain_dol_df["2022"] > rain_dol_df.loc[:, "2000":"2021"].max(axis=1)
] = True
rain_dol_22high = rain_dol_df["max_2021"].sum()
rain_dol_23high = rain_dol_df["max_2022"].sum()
print("Number of states for which 2022 rainy day fund balances")
print("are 23-year high is", rain_dol_23high, "states.")
print("Number of states for which 2021 rainy day fund balances")
print("are 22-year high is", rain_dol_22high, "states.")
print("Tennessee 2022:", rain_dol_df.loc["Tennessee","max_2022"])
print("Tennessee 2021:", rain_dol_df.loc["Tennessee","max_2021"])

Number of states for which 2022 rainy day fund balances
are 23-year high is 36 states.
Number of states for which 2021 rainy day fund balances
are 22-year high is 29 states.
Tennessee 2022: True
Tennessee 2021: True


/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/491360725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_dol_df["max_2021"][
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/491360725.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_dol_df["max_2022"][


In [10]:
# Number of states for which estimated 2022 rainy day fund balances
# as percent of general fund expenditures represent a 23-year high
rain_pct_df["max_2021"] = False
rain_pct_df["max_2022"] = False
rain_pct_df["max_2021"][
    rain_pct_df["2021"] > rain_pct_df.loc[:, "2000":"2020"].max(axis=1)
] = True
rain_pct_df["max_2022"][
    rain_pct_df["2022"] > rain_pct_df.loc[:, "2000":"2021"].max(axis=1)
] = True
rain_pct_22high = rain_pct_df["max_2021"].sum()
rain_pct_23high = rain_pct_df["max_2022"].sum()
print("")
print("Number of states for which 2022 rainy day fund balances")
print("as percent of general fund expenditures are 23-year high")
print("is", rain_pct_23high, "states.")
print("Number of states for which 2021 rainy day fund balances")
print("as percent of general fund expenditures are 22-year high")
print("is", rain_pct_22high, "states.")
print("Tennessee 2022:", rain_pct_df.loc["Tennessee","max_2022"])
print("Tennessee 2021:", rain_pct_df.loc["Tennessee","max_2021"])


Number of states for which 2022 rainy day fund balances
as percent of general fund expenditures are 23-year high
is 20 states.
Number of states for which 2021 rainy day fund balances
as percent of general fund expenditures are 22-year high
is 26 states.
Tennessee 2022: False
Tennessee 2021: True


/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/3481225494.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_pct_df["max_2021"] = False
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/3481225494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_pct_df["max_2022"] = False
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/3481225494.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

In [11]:
# Number of states for which estimated 2022 total balances and reserves
# represent a 23-year high
totbal_dol_df["max_2021"] = False
totbal_dol_df["max_2022"] = False
totbal_dol_df["max_2021"][
    totbal_dol_df["2021"] > totbal_dol_df.loc[:, "2000":"2020"].max(axis=1)
] = True
totbal_dol_df["max_2022"][
    totbal_dol_df["2022"] > totbal_dol_df.loc[:, "2000":"2021"].max(axis=1)
] = True
totbal_dol_22high = totbal_dol_df["max_2021"].sum()
totbal_dol_23high = totbal_dol_df["max_2022"].sum()
print("")
print("Number of states for which 2022 total balances and reserves")
print("are 23-year high is", totbal_dol_23high, "states.")
print("Number of states for which 2021 total balances and reserves")
print("are 22-year high is", totbal_dol_22high, "states.")
print("Tennessee 2022:", totbal_dol_df.loc["Tennessee","max_2022"])
print("Tennessee 2021:", totbal_dol_df.loc["Tennessee","max_2021"])


Number of states for which 2022 total balances and reserves
are 23-year high is 26 states.
Number of states for which 2021 total balances and reserves
are 22-year high is 42 states.
Tennessee 2022: False
Tennessee 2021: True


/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/503589125.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totbal_dol_df["max_2021"][
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/503589125.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totbal_dol_df["max_2022"][


In [12]:
# Number of states for which estimated 2022 total balances and reserves
# as percent of general fund expenditures represent a 23-year high
totbal_pct_df["max_2021"] = False
totbal_pct_df["max_2022"] = False
totbal_pct_df["max_2021"][
    totbal_pct_df["2021"] > totbal_pct_df.loc[:, "2000":"2020"].max(axis=1)
] = True
totbal_pct_df["max_2022"][
    totbal_pct_df["2022"] > totbal_pct_df.loc[:, "2000":"2021"].max(axis=1)
] = True
totbal_pct_22high = totbal_pct_df["max_2021"].sum()
totbal_pct_23high = totbal_pct_df["max_2022"].sum()
print("")
print("Number of states for which 2022 total balances and reserves")
print("as percent of general fund expenditures are 23-year high")
print("is", totbal_pct_23high, "states.")
print("Number of states for which 2021 total balances and reserves")
print("as percent of general fund expenditures are 22-year high")
print("is", totbal_pct_22high, "states.")
print("Tennessee 2022:", totbal_pct_df.loc["Tennessee","max_2022"])
print("Tennessee 2021:", totbal_pct_df.loc["Tennessee","max_2021"])


Number of states for which 2022 total balances and reserves
as percent of general fund expenditures are 23-year high
is 20 states.
Number of states for which 2021 total balances and reserves
as percent of general fund expenditures are 22-year high
is 32 states.
Tennessee 2022: False
Tennessee 2021: True


/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/1554983929.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totbal_pct_df["max_2021"] = False
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/1554983929.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totbal_pct_df["max_2022"] = False
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_39111/1554983929.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

## 2. Tennessee Business Tax Landscape
Important features of TN business tax system:
* 4 main components:
  1. "Excise Tax"
    * Essentailly a CIT
    * Rate is 6.5%
    * Depreciation handled under same rules as IRC *EXCEPT* no bonus depreciation.
  2. Franchise tax
    * Tax on the value of assets of the business
    * Rate is 0.26%
  3. Gross receipts tax
    * TVA
    * mixed drinks
    * vending machines
    * bottlers
  4. Business tax
    * Tax on the gross receipts of the business
    * Some industries (e.g., utilities) handled differently
    * Rate varies (see [here, p. 62](https://www.tn.gov/content/dam/tn/revenue/documents/tax_manuals/august-2022/Business-Tax.pdf))
      * Varies between 0.02% and 0.3% (depends on industry)

### Table 2. Tennessee business taxes: dollars and percent of total tax revenue

In [13]:
# Read in the Tennessee Department of Revenue monthly receipts
# data, June 2022, worksheets Pg6 and Pg8
main_path = (
    "https://github.com/TheCGO/TN-BusinessTax/blob/main/data/" +
    "Main202206.xlsx?raw=true"
)
tn_Jun2022_Pg6_df = pd.read_excel(
    main_path,
    sheet_name="Pg6",
    header=3,
    index_col=1,
)
tn_Jun2022_Pg6_df.drop(
    ['Unnamed: 0', 'GAIN OR LOSS', 'PERCENT'], axis=1, inplace=True
)
tn_Jun2022_Pg6_df.rename(
    columns={" FY 2021": "FY 2021", " FY 2022": "FY 2022"}, inplace=True
)
tn_Jun2022_Pg8_df = pd.read_excel(
    main_path,
    sheet_name="Pg8",
    header=3,
    index_col=1,
)
tn_Jun2022_Pg8_df.drop(
    ['Unnamed: 0', 'GAIN OR LOSS', 'PERCENT'], axis=1, inplace=True
)
tn_Jun2022_Pg8_df.rename(
    columns={" FY 2021": "FY 2021"}, inplace=True
)

# Get statistics for Tennessee excise tax
ex_2021 = tn_Jun2022_Pg6_df.loc["11501-11503 Excise", "FY 2021"]
fr_2021 = tn_Jun2022_Pg6_df.loc["11401-11403 Franchise", "FY 2021"]
FE_est_pmt_2021 = tn_Jun2022_Pg6_df.loc["12101 F&E Est Payments", "FY 2021"]
tot_rev_2021 = tn_Jun2022_Pg8_df.loc["GRAND TOTAL", "FY 2021"]
extax_2021_dol = ex_2021 + (ex_2021 / (ex_2021 + fr_2021)) * FE_est_pmt_2021
extax_2021_pct = extax_2021_dol / tot_rev_2021

ex_2022 = tn_Jun2022_Pg6_df.loc["11501-11503 Excise", "FY 2022"]
fr_2022 = tn_Jun2022_Pg6_df.loc["11401-11403 Franchise", "FY 2022"]
FE_est_pmt_2022 = tn_Jun2022_Pg6_df.loc["12101 F&E Est Payments", "FY 2022"]
tot_rev_2022 = tn_Jun2022_Pg8_df.loc["GRAND TOTAL", "FY 2022"]
extax_2022_dol = ex_2022 + (ex_2022 / (ex_2022 + fr_2022)) * FE_est_pmt_2022
extax_2022_pct = extax_2022_dol / tot_rev_2022

print("Tennessee excise tax 2021 $:", extax_2021_dol)
print("Tennessee excise tax 2021 % tot revenue:", extax_2021_pct)
print("Tennessee excise tax 2022 $:", extax_2022_dol)
print("Tennessee excise tax 2022 % tot revenue:", extax_2022_pct)

# Get statistics for Tennessee franchise tax
frtax_2021_dol = fr_2021 + (fr_2021 / (ex_2021 + fr_2021)) * FE_est_pmt_2021
frtax_2021_pct = frtax_2021_dol / tot_rev_2021

frtax_2022_dol = fr_2022 + (fr_2022 / (ex_2022 + fr_2022)) * FE_est_pmt_2022
frtax_2022_pct = frtax_2022_dol / tot_rev_2022

print("")
print("Tennessee franchise tax 2021 $:", frtax_2021_dol)
print("Tennessee franchise tax 2021 % tot revenue:", frtax_2021_pct)
print("Tennessee franchise tax 2022 $:", frtax_2022_dol)
print("Tennessee franchise tax 2022 % tot revenue:", frtax_2022_pct)

# Get statistics for Tennessee gross receipts tax
TVA_2021 = tn_Jun2022_Pg8_df.loc["TVA IN LIEU", "FY 2021"]
bot1_2021 = tn_Jun2022_Pg8_df.loc["10801 Bottlers", "FY 2021"]
gwpl_2021 = tn_Jun2022_Pg8_df.loc["10802 Gas, Water, Power & Light", "FY 2021"]
mbc_2021 = tn_Jun2022_Pg8_df.loc["10803 Mixing Bars, Clubs, etc.", "FY 2021"]
vm_2021 = tn_Jun2022_Pg8_df.loc["10809 Vending Machines", "FY 2021"]
bot2_2021 = tn_Jun2022_Pg8_df.loc["10810 Bottlers", "FY 2021"]
gr_other_2021 = bot1_2021 + gwpl_2021 + mbc_2021 + vm_2021 + bot2_2021
grtax_2021_dol = TVA_2021 + gr_other_2021
grtax_2021_pct = grtax_2021_dol / tot_rev_2021

TVA_2022 = tn_Jun2022_Pg8_df.loc["TVA IN LIEU", "FY 2022"]
bot1_2022 = tn_Jun2022_Pg8_df.loc["10801 Bottlers", "FY 2022"]
gwpl_2022 = tn_Jun2022_Pg8_df.loc["10802 Gas, Water, Power & Light", "FY 2022"]
mbc_2022 = tn_Jun2022_Pg8_df.loc["10803 Mixing Bars, Clubs, etc.", "FY 2022"]
vm_2022 = tn_Jun2022_Pg8_df.loc["10809 Vending Machines", "FY 2022"]
bot2_2022 = tn_Jun2022_Pg8_df.loc["10810 Bottlers", "FY 2022"]
gr_other_2022 = bot1_2022 + gwpl_2022 + mbc_2022 + vm_2022 + bot2_2022
grtax_2022_dol = TVA_2022 + gr_other_2022
grtax_2022_pct = grtax_2022_dol / tot_rev_2022

print("")
print("Tennessee gross receipts tax 2021 $:", grtax_2021_dol)
print("Tennessee gross receipts tax 2021 % tot revenue:", grtax_2021_pct)
print("Tennessee gross receipts tax 2022 $:", grtax_2022_dol)
print("Tennessee gross receipts tax 2022 % tot revenue:", grtax_2022_pct)

# Get statistics for Tennessee business tax (excluding city and county taxes)
st_tax_2021 = tn_Jun2022_Pg6_df.loc["12003 State Tax", "FY 2021"]
cls_tax_2021 = tn_Jun2022_Pg6_df.loc["12006-12009-Class 1-4", "FY 2021"]
cls5_tax_2021 = tn_Jun2022_Pg6_df.loc["12010 Class 5 Industrial  Loan & Thrift", "FY 2021"]
trs_tax_2021 = tn_Jun2022_Pg6_df.loc["12011 Transient Vendor, Flea Mkt & Other", "FY 2021"]
aud_tax_2021 = tn_Jun2022_Pg6_df.loc["12012  Audit P & I", "FY 2021"]
vol_tax_2021 = tn_Jun2022_Pg6_df.loc["12013 Voluntary Disclosure State", "FY 2021"]
btax_2021_dol = (
    st_tax_2021 + cls_tax_2021 + cls5_tax_2021 + trs_tax_2021 +
    aud_tax_2021 + vol_tax_2021
)
btax_2021_pct = btax_2021_dol / tot_rev_2021

st_tax_2022 = tn_Jun2022_Pg6_df.loc["12003 State Tax", "FY 2022"]
cls_tax_2022 = tn_Jun2022_Pg6_df.loc["12006-12009-Class 1-4", "FY 2022"]
cls5_tax_2022 = tn_Jun2022_Pg6_df.loc["12010 Class 5 Industrial  Loan & Thrift", "FY 2022"]
trs_tax_2022 = tn_Jun2022_Pg6_df.loc["12011 Transient Vendor, Flea Mkt & Other", "FY 2022"]
aud_tax_2022 = tn_Jun2022_Pg6_df.loc["12012  Audit P & I", "FY 2022"]
vol_tax_2022 = tn_Jun2022_Pg6_df.loc["12013 Voluntary Disclosure State", "FY 2022"]
btax_2022_dol = (
    st_tax_2022 + cls_tax_2022 + cls5_tax_2022 + trs_tax_2022 +
    aud_tax_2022 + vol_tax_2022
)
btax_2022_pct = btax_2022_dol / tot_rev_2022

print("")
print("Tennessee business tax 2021 $:", btax_2021_dol)
print("Tennessee business tax 2021 % tot revenue:", btax_2021_pct)
print("Tennessee business tax 2022 $:", btax_2022_dol)
print("Tennessee business tax 2022 % tot revenue:", btax_2022_pct)

Tennessee excise tax 2021 $: 2564304204.311518
Tennessee excise tax 2021 % tot revenue: 0.13953139426917469
Tennessee excise tax 2022 $: 3016322556.778461
Tennessee excise tax 2022 % tot revenue: 0.1443310576963863

Tennessee franchise tax 2021 $: 1381839681.1384816
Tennessee franchise tax 2021 % tot revenue: 0.07518999385546422
Tennessee franchise tax 2022 $: 1518181589.8615394
Tennessee franchise tax 2022 % tot revenue: 0.07264500082972758

Tennessee gross receipts tax 2021 $: 374465076.61
Tennessee gross receipts tax 2021 % tot revenue: 0.02037575501247324
Tennessee gross receipts tax 2022 $: 368503143.53999996
Tennessee gross receipts tax 2022 % tot revenue: 0.017632878271605163

Tennessee business tax 2021 $: 263550852.28000003
Tennessee business tax 2021 % tot revenue: 0.014340583234090565
Tennessee business tax 2022 $: 282713361.26
Tennessee business tax 2022 % tot revenue: 0.013527836525261017


## 3. Investment Incentives using Cost of Capital Calculator
Computing the cost-of-capital at the state-level:
  * the statutory rate used will be the sum of the federal and state CIT
    * But can deduct state taxes at federal level
    * So maybe rate is Fed Rate + State rate - fed rate * state rate?
  * franchise tax can use the wealth tax parameter in CCC
  * The interaction of state and federal is not something I want to rush into CCC, so we'll do most of the calculations here by functions defined in this notebook (with a bit of assistance from a couple CCC functions.


Equation for the cost of capital at when considering state taxes in TN:
$$
\rho = \left[\frac{(r-\pi+\delta)(1-k-u^fz^f-u^sz^s + u^fu^sz^s)}{(1-u^f-u^s+u^fu^s)} + w \right] / (1-\tau^{GR}) - \delta
$$
where:
* $\rho$ = cost of capital
* $r$ = nominal discount rate
* $\pi$ = inflation rate
* $k$ = investment tax credit rate
* $u^f$ = statutory CIT rate at the federal level 
* $u^s$ = statutory CIT rate at the state level 
* $z^f$ = NPV of depreciation deduction under federal system
* $z^s$ = NPV of depreciation deduction under state system
* $w$ = property tax rate
* $\tau^{GR}$ = gross receipts tax rate

Calibration:

| Parameter     | Value | Source |
| ----------- | ----------- |----------- |
| $r$     | 0.06      | To give real return of 4\%|
| $\pi$  | 0.02      | Fed target|
| $k$  | 0.0        | No general invest tax credit at federal or state level|
| $u^f$  | 0.21       | Federal IRC|
| $u^s$  | 0.065       | Excise tax rate in TN law|
| $z^f$  | Varies        | Federal IRC|
| $z^s$  |   Varies      | Like Federal IRC, but no bonus deprec|
| $w$  | 0.0026       | Franchise tax rate in TN law |
| $\tau^{GR}$  | 0.0002-0.003       | Bus tax rate in TN law|

In [2]:
!pip install cost-of-capital-calculator &> /dev/null
!pip install taxcalc &> /dev/null
!pip install requests &> /dev/null

Now we'll import the Python packages and modules we'll work with:

In [3]:
# imports
import ccc
import taxcalc as tc
import numpy as np
import pandas as pd
import requests
import plotly.express as px
import plotly.io as pio
# set template for plotting
pio.templates.default = "plotly_white"

### Setup

Create objects to work with

In [4]:
# Creating an instance of the Assets class and call it "assets"
assets = ccc.data.Assets()
# Create an instance of the DepreciationParams
# class object and call it dp
dp = ccc.parameters.DepreciationParams()

In [5]:
# define new cost of capital function to take into account state taxes
def eq_coc_state(delta, z_f, z_s, w, u_f, u_s, tau_GR, inv_tax_credit, pi, r):
    r'''
    Compute the cost of capital
    .. math::
        \rho = \frac{\frac{(r-\pi+\delta)}{1-u^f-u^s+u^fu^s}(1-u^fz^f-u^sz^s+u^fu^sz^s)+w}{(1-\tau^GR)}-\delta
    Args:
        delta (array_like): rate of economic depreciation
        z_f (array_like): net present value of depreciation deductions for
            $1 of investment under the federal system
        z_s (array_like): net present value of depreciation deductions for
            $1 of investment under the state system
        w (scalar): property tax rate
        u_f (scalar): statutory marginal tax rate for the first layer of
            income taxes at the federal level
        u_s (scalar): statutory marginal tax rate for the first layer of
            income taxes at the state level
        tau_GR (scalar): gross receipts tax
        inv_tax_credit (scalar): investment tax credit rate
        pi (scalar): inflation rate
        r (scalar): discount rate
    Returns:
        rho (array_like): the cost of capital
    '''
    rho = (
        ((((r - pi + delta) / (1 - u_f - u_s + u_f * u_s)) *
          (1 - inv_tax_credit - u_f * z_f - u_s * z_s + u_f * u_s * z_s) + w) /
         (1 - tau_GR)) - delta
    )

    return rho

In [6]:
# Define model parameters
inflation_rate = 0.02 # Inflation rate
nominal_int_rate = 0.06
fraction_financed_w_debt = 0.0
federal_bonus_depreciation = {"machines": 0.8, "buildings": 0.0, "intangibles": 0.8}
depreciation_rates = {"machines": 0.1031, "buildings": 0.0314, "intangibles": 0.33}
# Example machine: EI40, example building: SI00, example intangible: ENS3
depreciation_lives = {"machines": 7, "buildings": 39, "intangibles": 3}
depreciation_methods = {"machines": "dbsl", "buildings": "sl", "intangibles": "sl"}
E = 0.06  # Expected after-tax return on corporate equity
profit_rate = 0.2 # 0.2 is a 20% profit rate
int_haircut = 0.0
u_f = 0.21
u_s = 0.065
franchise_tax_rate = 0.0026
tau_GR = 0.003 # this is the higher end 0.0002-0.003 is what TF reports, varies by industry
inv_tax_credit = 0.0
bonus_s = {"machines": 0.0, "buildings": 0.0, "intangibles": 0.0}


In [7]:
# compute outputs
def compute_outputs(u_s, bonus_s, franchise_tax_rate, tau_GR):
    """
    This function computes the outputs of interest and allows one to change the
    parameters of TN state law

    Args:
      u_s (scalar): statutory marginal tax rate for the first layer of
            income taxes at the state level
      bonus_s (dict): rates of bonus depreciation in TN by asset type
      franchise_tax_rate (scalar): TN franchise tax
      tau_GR (scalar): gross receipts tax
    """
    out_dict = {"machines": {}, "buildings": {}, "intangibles": {}}
    for k, v in depreciation_rates.items():
        r = ccc.paramfunctions.calc_r(
            u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
            nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
        r_prime = ccc.paramfunctions.calc_r_prime(
            nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
        if depreciation_methods[k] == "dbsl":
            z_f = ccc.calcfunctions.dbsl(depreciation_lives[k], 2, federal_bonus_depreciation[k], r)
            z_s = ccc.calcfunctions.dbsl(depreciation_lives[k], 2, bonus_s[k], r)
        elif depreciation_methods[k] == "sl":
            z_f = ccc.calcfunctions.sl(depreciation_lives[k], federal_bonus_depreciation[k], r)
            z_s = ccc.calcfunctions.sl(depreciation_lives[k], bonus_s[k], r)
        else:
          print("Please enter one of: dbsl, sl")
          assert False
        rho = eq_coc_state(
                depreciation_rates[k], z_f, z_s, franchise_tax_rate,
                u_f, u_s, tau_GR, inv_tax_credit, inflation_rate, r)
        metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
        eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
        out_dict[k]["rho"] = rho
        out_dict[k]["metr"] = metr
        out_dict[k]["eatr"] = eatr
    return out_dict

In [8]:
base_df = pd.DataFrame(compute_outputs(u_s, bonus_s, franchise_tax_rate, tau_GR))
repealGR_df = pd.DataFrame(compute_outputs(u_s, bonus_s, franchise_tax_rate, 0.0))
fedbonus_df = pd.DataFrame(compute_outputs(u_s, federal_bonus_depreciation, franchise_tax_rate, tau_GR))
repealFT_df = pd.DataFrame(compute_outputs(u_s, bonus_s, 0.0, tau_GR))

In [9]:
# Create policy names and put in one dataframe
base_df['Policy'] = "Current Law"
repealGR_df['Policy'] = "Repeal Business Tax"
fedbonus_df['Policy'] = "Follow Federal Bonus Depreciation"
repealFT_df['Policy'] = "Repeal Franchise Tax"
# append dataframes together
df = pd.concat([base_df, repealGR_df, fedbonus_df, repealFT_df])
df.reset_index(inplace=True)
df.rename(columns={"index": "output_var"}, inplace=True)
df = pd.melt(df, id_vars=["Policy", "output_var"], var_name="asset_type")

In [10]:
# Plot results
fig = px.bar(df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")], x="Policy", y="value",
             color='Policy', labels={'value':'Marginal Effective Tax Rate'}, height=400)
fig.show()

In [11]:
# Plot results
fig = px.bar(df[(df["output_var"]=="rho") & (df["asset_type"]=="machines")], x="Policy", y="value",
             color='Policy', labels={'value':'Cost of Capital'}, height=400)
fig.show()

In [12]:
fig = px.bar(df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")], x="value", y="Policy",
             color='Policy', labels={'value':'Marginal Effective Tax Rate'}, height=400)
fig.show()

In [13]:
fig = px.bar(df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")], x="Policy", y="value",
             color='Policy', labels={'value':'Marginal Effective Tax Rate'}, height=400)
fig.show()

In [17]:
# Plot results
fig = px.histogram(df[(df["output_var"]=="metr") & (df['Policy'].isin(["Current Law", "Repeal Business Tax", "Follow Federal Bonus Depreciation", "Repeal Franchise Tax"]))], x="asset_type", y="value",
             color='Policy', barmode='group', labels={'asset_type':'Asset Type', 'value': 'Marginal Effective Tax Rate'},
             height=400)
fig.show()

In [15]:
# compute example results for equipment
r = ccc.paramfunctions.calc_r(
        u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
        nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
r_prime = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
z_f = ccc.calcfunctions.dbsl(5, 2, 0.8, r)  # 5 year asset, double declining balance, 80% bonus
z_s = ccc.calcfunctions.dbsl(5, 2, 0.0, r)  # 5 year asset, double declining balance, no bonus
rho = eq_coc_state(
        depreciation_rates['machines'], z_f, z_s, franchise_tax_rate,
        u_f, u_s, tau_GR, inv_tax_credit, inflation_rate, r)
metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
print('Machines: rho = ', rho, ', metr = ', metr, ' and eatr = ', eatr)

Machines: rho =  0.06608968747029248 , metr =  0.09214277905353714  and eatr =  0.3025207148336957


In [16]:
# compare to federal
r = ccc.paramfunctions.calc_r(
        u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
        nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
r_prime = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
z_f = ccc.calcfunctions.dbsl(5, 2, 0.8, r)  # 5 year asset, double declining balance, 80% bonus
rho = ccc.calcfunctions.eq_coc(
        depreciation_rates['machines'], z_f, 0.0,
        u_f, inv_tax_credit, inflation_rate, r)
metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
print('Machines: rho = ', rho, ', metr = ', metr, ' and eatr = ', eatr)

Machines: rho =  0.06125450230184801 , metr =  0.020480164799415276  and eatr =  0.28816867645746036
